In [1]:
import os
import sys
sys.path.append(r'E:\毕设\work\utils')
import numpy
import argparse
import sys
import soundfile
import numpy as np
import librosa
import h5py
import time
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import logging

import config
from utilities import calculate_scalar, create_folder, read_audio

In [2]:
dataset_dir = "E:\\毕设"
workspace = "E:\\毕设\\work"
data_type = "development"
mini_data = False

sample_rate = config.sample_rate
window_size = config.window_size
overlap = config.overlap

def calculate_logmel(audio_path, sample_rate, n, audio_cuts):

    (audio, _) = read_audio(audio_path, target_fs=sample_rate)
    
    [cut1,cut2] = audio_cuts[n].split(',')
    audio = audio[int(float(cut1)*sample_rate):int(float(cut2)*sample_rate)]

    audio = audio / np.max(np.abs(audio))
    
    S = np.abs(librosa.stft(audio))
    feature = librosa.feature.spectral_contrast(S=S, sr=sample_rate, n_bands=7, fmin=100, n_fft=window_size,hop_length=overlap).T

    return feature

def ExtractSC(data_type):
    if data_type == 'development':
            audio_dir = dataset_dir
            meta_csv = os.path.join(workspace, 'train.xlsx')

    elif data_type == 'test':
        audio_dir = dataset_dir
        meta_csv = os.path.join(workspace, 'test.xlsx')
        
    hdf5_path = os.path.join(workspace, 'features', 'logmel', 
                             '{}.h5'.format(data_type))
        
    create_folder(os.path.dirname(hdf5_path))
    df = pd.DataFrame(pd.read_excel(meta_csv))
    
    audio_names = []
    audio_cuts = []

    if data_type == 'development':
        manually_verifications = []
        target_labels = []

    for row in df.iterrows():

        audio_name = row[1]['name']
        audio_names.append(audio_name)
        audio_cut = row[1]['cut']
        audio_cuts.append(audio_cut)

        if data_type == 'development':

            target_label = row[1]['label']
            manually_verification = row[1]['manually_verification']

            target_labels.append(target_label)
            manually_verifications.append(manually_verification)
        
    hf = h5py.File(hdf5_path, 'w')

    hf.create_dataset(
        name='feature',
        shape=(0, 8),
        maxshape=(None, 8),
        dtype=np.float32)

    calculate_time = time.time()
    bgn_fin_indices = []
    
    for (n, audio_name) in enumerate(audio_names):

        print(n, audio_name)

        audio_path = os.path.join(audio_dir, audio_name)
        feature = calculate_logmel(audio_path, sample_rate, n, audio_cuts)

        print(feature.shape)

        # Write feature to hdf5
        bgn_indice = hf['feature'].shape[0]
        fin_indice = bgn_indice + feature.shape[0]

        hf['feature'].resize((fin_indice, 8))
        hf['feature'][bgn_indice: fin_indice] = feature

        bgn_fin_indices.append((bgn_indice, fin_indice))
        
    # Write infos to hdf5
    hf.create_dataset(name='filename', 
                      data=[s.encode() for s in audio_names], 
                      dtype='S32')

    hf.create_dataset(name='bgn_fin_indices',
                      data=bgn_fin_indices,
                      dtype=np.int32)

    if data_type == 'development':

        hf.create_dataset(name='label', 
                          data=[s.encode() for s in target_labels], 
                          dtype='S32')

        hf.create_dataset(name='manually_verification',
                          data=manually_verifications,
                          dtype=np.int32)

    hf.close()

    print("Write out hdf5 file to {}".format(hdf5_path))
    print("Time spent: {} s".format(time.time() - calculate_time))

ExtractSC("development")

0 乐器音频/三弦T0289/T0289A2-1c2-1.wav
(130, 8)
1 乐器音频/五音排鼓D0252/D0252A2-1e1.wav
(165, 8)
2 乐器音频/中音笙C0263/C0263A2-1c3-1.wav
(132, 8)
3 乐器音频/中音板胡L0240/L0240A2-1c4-1.wav
(77, 8)
4 乐器音频/箜篌T0254/T0254A2-1c2-1.wav
(112, 8)
5 乐器音频/葫芦琴L0314/L0314A2-1c2-2.wav
(441, 8)
6 乐器音频/箫C0282/C0282A2-1d3-1.wav
(46, 8)
7 乐器音频/三弦T0289/T0289A2-1c2-1.wav
(52, 8)
8 乐器音频/二胡L0266/L0266A2-1c3-1.wav
(134, 8)
9 乐器音频/扬琴T0267/T0267A2-1c3-1.wav
(99, 8)
10 乐器音频/葫芦丝C0309/C0309A2-1d7-2.wav
(150, 8)
11 乐器音频/澜沧小三弦T0300/T0300A2-1e2.wav
(32, 8)
12 乐器音频/编钟D0241/D0241A2-1c4-2.wav
(112, 8)
13 乐器音频/雷琴L0256/L0256A2-1c3-2.wav
(107, 8)
14 乐器音频/中音板胡L0240/L0240A2-1c1-2.wav
(111, 8)
15 乐器音频/柳琴T0261/T0261A2-1c4-1.wav
(40, 8)
16 乐器音频/芦笙C0307/C0307A2-1c4-1.wav
(450, 8)
17 乐器音频/小闷笛C0303/C0303A2-1c1-1.wav
(249, 8)
18 乐器音频/南音三弦T0295/T0295A2-1e1.wav
(46, 8)
19 乐器音频/三弦T0289/T0289A2-1d3-1.wav
(32, 8)
20 乐器音频/柳琴T0261/T0261A2-1c2-1.wav
(224, 8)
21 乐器音频/G调梆笛C0237/C0237A2-1c2-1.wav
(41, 8)
22 乐器音频/古筝T0255/T0255A2-1c3-2.wav
(104, 8)
23 乐器音频/低音加键唢呐C0257/

(185, 8)
190 乐器音频/G调梆笛C0237/C0237A2-1d4-2.wav
(63, 8)
191 乐器音频/吐良C0308/C0308A2-1c3-1.wav
(150, 8)
192 乐器音频/南音琵琶T0294/T0294A2-1c3-1.wav
(35, 8)
193 乐器音频/三弦T0289/T0289A2-1c3-2.wav
(117, 8)
194 乐器音频/A调曲笛C0280/C0280A2-1d3-1.wav
(240, 8)
195 乐器音频/玎T0301/T0301A2-1d3-1.wav
(70, 8)
196 乐器音频/高音板胡L0239/L0239A2-1d6-2.wav
(103, 8)
197 乐器音频/三弦T0289/T0289A2-1c4-2.wav
(119, 8)
198 乐器音频/牛角琴L0312/L0312A2-1d4-1.wav
(26, 8)
199 乐器音频/柳琴T0261/T0261A2-1c2-2.wav
(69, 8)
200 乐器音频/唢呐C0296/C0296A2-1c1-2.wav
(205, 8)
201 乐器音频/柳琴T0261/T0261A2-1d3-1.wav
(37, 8)
202 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(223, 8)
203 乐器音频/高音键笙C0243/C0243A2-1c3-1.wav
(217, 8)
204 乐器音频/唢呐C0296/C0296A2-1d9-2.wav
(415, 8)
205 乐器音频/G调梆笛C0237/C0237A2-1d1-2.wav
(404, 8)
206 乐器音频/雷琴L0256/L0256A2-1c2-2.wav
(114, 8)
207 乐器音频/二弦L0285/L0285A2-1e2.wav
(184, 8)
208 乐器音频/编铓D0298/D0298A2-1e1.wav
(135, 8)
209 乐器音频/扬琴T0267/T0267A2-1d2-1.wav
(410, 8)
210 乐器音频/五音排鼓D0252/D0252A2-1e1.wav
(164, 8)
211 乐器音频/六角高胡L0292/L0292A2-1c2-1.wav
(59, 8)
212 乐器音频/高音键笙C0243

(154, 8)
377 乐器音频/芦笙C0307/C0307A2-1d1-1.wav
(380, 8)
378 乐器音频/巴乌C0310/C0310A2-1d3-1.wav
(25, 8)
379 乐器音频/箜篌T0254/T0254A2-1e3.wav
(299, 8)
380 乐器音频/澜沧小三弦T0300/T0300A2-1e2.wav
(29, 8)
381 乐器音频/中阮T0260/T0260A2-1c4-2.wav
(93, 8)
382 乐器音频/竹排琴D0315/D0315A2-1c1-1.wav
(43, 8)
383 乐器音频/南音二弦L0293/L0293A2-1c1-1.wav
(237, 8)
384 乐器音频/唢呐C0296/C0296A2-1c1-1.wav
(150, 8)
385 乐器音频/吐良C0308/C0308A2-1d1-1.wav
(138, 8)
386 乐器音频/六角高胡L0292/L0292A2-1b3-1.wav
(56, 8)
387 乐器音频/中阮T0260/T0260A2-1b1-2.wav
(1090, 8)
388 乐器音频/中阮T0260/T0260A2-1c3-2.wav
(344, 8)
389 乐器音频/古筝T0255/T0255A2-1c4-2.wav
(199, 8)
390 乐器音频/传统笙C0244/C0244A2-1c3-1.wav
(99, 8)
391 乐器音频/草帽镲D0268/D0268A2-1d3-1.wav
(25, 8)
392 乐器音频/高音板胡L0239/L0239A2-1c3-1.wav
(114, 8)
393 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(182, 8)
394 乐器音频/三弦T0289/T0289A2-1c3-1.wav
(75, 8)
395 乐器音频/编钟D0241/D0241A2-1c5-2.wav
(79, 8)
396 乐器音频/二胡L0266/L0266A2-1c3-1.wav
(159, 8)
397 乐器音频/柳琴T0261/T0261A2-1e2.wav
(71, 8)
398 乐器音频/管子C0265/C0265A2-1c2-1.wav
(87, 8)
399 乐器音频/二胡L0266/L0266A2-

(494, 8)
564 乐器音频/G调梆笛C0237/C0237A2-1d4-2.wav
(69, 8)
565 乐器音频/高音键笙C0243/C0243A2-1c5-1.wav
(160, 8)
566 乐器音频/管子C0265/C0265A2-1c1-1.wav
(87, 8)
567 乐器音频/小锣D0275/D0275A2-1e1.wav
(55, 8)
568 乐器音频/唢呐C0296/C0296A2-1c4-2.wav
(44, 8)
569 乐器音频/南音三弦T0295/T0295A2-1e1.wav
(46, 8)
570 乐器音频/牛角琴L0312/L0312A2-1c2-1.wav
(356, 8)
571 乐器音频/传统笙C0244/C0244A2-1c5-2.wav
(76, 8)
572 乐器音频/箜篌T0254/T0254A2-1c4-1.wav
(56, 8)
573 乐器音频/傈僳族奇奔T0302/T0302A2-1e1.wav
(153, 8)
574 乐器音频/牛角琴L0312/L0312A2-1d6-2.wav
(222, 8)
575 乐器音频/扬琴T0267/T0267A2-1c4-1.wav
(308, 8)
576 乐器音频/三弦T0289/T0289A2-1c4-1.wav
(122, 8)
577 乐器音频/中虎D0273/D0273A2-1e1.wav
(22, 8)
578 乐器音频/葫芦琴L0314/L0314A2-1d3-1.wav
(22, 8)
579 乐器音频/中音加键唢呐C0259/C0259A2-1c3-1.wav
(84, 8)
580 乐器音频/G调新笛C0281/C0281A2-1d9-1.wav
(288, 8)
581 乐器音频/雷琴L0256/L0256A2-1c5-2.wav
(81, 8)
582 乐器音频/G调梆笛C0237/C0237A2-1c5-1.wav
(262, 8)
583 乐器音频/G调梆笛C0237/C0237A2-1c4-2.wav
(323, 8)
584 乐器音频/三弦T0289/T0289A2-1c1-2.wav
(104, 8)
585 乐器音频/扁八角高胡L0291/L0291A2-1c1-2.wav
(89, 8)
586 乐器音频/二胡L0266/

(69, 8)
754 乐器音频/编铓D0298/D0298A2-1e1.wav
(134, 8)
755 乐器音频/柳琴T0261/T0261A2-1e2.wav
(155, 8)
756 乐器音频/二胡L0266/L0266A2-1d10-1.wav
(69, 8)
757 乐器音频/中音笙C0263/C0263A2-1d5-1.wav
(226, 8)
758 乐器音频/牛角琴L0312/L0312A2-1d3-2.wav
(50, 8)
759 乐器音频/三弦T0289/T0289A2-1d3-1.wav
(63, 8)
760 乐器音频/三弦T0289/T0289A2-1c3-1.wav
(184, 8)
761 乐器音频/葫芦丝C0309/C0309A2-1c1-1.wav
(441, 8)
762 乐器音频/扁八角高胡L0291/L0291A2-1c2-1.wav
(56, 8)
763 乐器音频/编磬D0242/D0242A2-1c5-2.wav
(77, 8)
764 乐器音频/高音板胡L0239/L0239A2-1c2-2.wav
(117, 8)
765 乐器音频/高音键笙C0243/C0243A2-1c2-1.wav
(129, 8)
766 乐器音频/二弦L0285/L0285A2-1e4.wav
(116, 8)
767 乐器音频/管子C0265/C0265A2-1e2.wav
(73, 8)
768 乐器音频/中胡L0297/L0297A2-1c4-1.wav
(83, 8)
769 乐器音频/中音加键唢呐C0259/C0259A2-1b1-2.wav
(373, 8)
770 乐器音频/扬琴T0267/T0267A2-1c1-2.wav
(429, 8)
771 乐器音频/牛角琴L0312/L0312A2-1c3-2.wav
(226, 8)
772 乐器音频/中音板胡L0240/L0240A2-1c1-1.wav
(113, 8)
773 乐器音频/传统笙C0244/C0244A2-1c1-2.wav
(103, 8)
774 乐器音频/南音琵琶T0294/T0294A2-1e1.wav
(25, 8)
775 乐器音频/牛角琴L0312/L0312A2-1c2-1.wav
(356, 8)
776 乐器音频/二弦L0285/L02

...

(346, 8)
14255 乐器音频/古筝T0255/T0255A2-1c3-1.wav
(112, 8)
14256 乐器音频/椰胡L0288/L0288A2-1e1.wav
(119, 8)
14257 乐器音频/扬琴T0267/T0267A2-1d1-1.wav
(280, 8)
14258 乐器音频/中音加键唢呐C0259/C0259A2-1e2.wav
(79, 8)
14259 乐器音频/深波D0287/D0287A2-1d2-2.wav
(42, 8)
14260 乐器音频/扬琴T0267/T0267A2-1c4-1.wav
(25, 8)
14261 乐器音频/竹排琴D0315/D0315A2-1c3-2.wav
(22, 8)
14262 乐器音频/抄锣D0272/D0272A2-1d2-1.wav
(256, 8)
14263 乐器音频/中阮T0260/T0260A2-1c3-1.wav
(240, 8)
14264 乐器音频/二胡L0266/L0266A2-1c2-2.wav
(87, 8)
14265 乐器音频/扁八角高胡L0291/L0291A2-1c3-1.wav
(63, 8)
14266 乐器音频/独弦琴T0317/T0317A2-1e2.wav
(622, 8)
14267 乐器音频/三弦T0289/T0289A2-1d10-1.wav
(164, 8)
14268 乐器音频/箜篌T0254/T0254A2-1c2-2.wav
(117, 8)
14269 乐器音频/中音笙C0263/C0263A2-1c1-1.wav
(145, 8)
14270 乐器音频/中阮T0260/T0260A2-1c1-1.wav
(111, 8)
14271 乐器音频/唢呐C0296/C0296A2-1d11-1.wav
(222, 8)
14272 乐器音频/高音板胡L0239/L0239A2-1c4-1.wav
(74, 8)
14273 乐器音频/二胡L0266/L0266A2-1c3-1.wav
(105, 8)
14274 乐器音频/编钟D0241/D0241A2-1c5-1.wav
(79, 8)
14275 乐器音频/扬琴T0267/T0267A2-1c1-2.wav
(184, 8)
14276 乐器音频/琵琶T0262/T0262A

(87, 8)
14435 乐器音频/吐良C0308/C0308A2-1d2-1.wav
(25, 8)
14436 乐器音频/高音键笙C0243/C0243A2-1c2-1.wav
(162, 8)
14437 乐器音频/葫芦丝C0309/C0309A2-1c3-1.wav
(373, 8)
14438 乐器音频/二胡L0266/L0266A2-1c3-2.wav
(87, 8)
14439 乐器音频/琵琶T0262/T0262A2-1c1-2.wav
(88, 8)
14440 乐器音频/南音琵琶T0294/T0294A2-1e1.wav
(26, 8)
14441 乐器音频/管子C0265/C0265A2-1e4.wav
(315, 8)
14442 乐器音频/扬琴T0267/T0267A2-1d2-1.wav
(50, 8)
14443 乐器音频/扬琴T0267/T0267A2-1c3-1.wav
(348, 8)
14444 乐器音频/编钟D0241/D0241A2-1c3-1.wav
(112, 8)
14445 乐器音频/扁八角高胡L0291/L0291A2-1c1-1.wav
(79, 8)
14446 乐器音频/三弦T0289/T0289A2-1c3-2.wav
(19, 8)
14447 乐器音频/傈僳族奇奔T0302/T0302A2-1c2-2.wav
(116, 8)
14448 乐器音频/中音板胡L0240/L0240A2-1e1.wav
(87, 8)
14449 乐器音频/低音笙C0264/C0264A2-1c3-2.wav
(226, 8)
14450 乐器音频/扬琴T0267/T0267A2-1d1-1.wav
(53, 8)
14451 乐器音频/扬琴T0267/T0267A2-1c1-2.wav
(371, 8)
14452 乐器音频/二胡L0266/L0266A2-1c1-2.wav
(100, 8)
14453 乐器音频/拉祜族葫芦笙C0306/C0306A2-1c1-1.wav
(362, 8)
14454 乐器音频/箫C0282/C0282A2-1c1-1.wav
(120, 8)
14455 乐器音频/中国大鼓D0248/D0248A2-1d9-1.wav
(19, 8)
14456 乐器音频/傈僳族奇奔T0302/T

(447, 8)
14614 乐器音频/编钟D0241/D0241A2-1c4-1.wav
(112, 8)
14615 乐器音频/三弦T0289/T0289A2-1c4-1.wav
(49, 8)
14616 乐器音频/中阮T0260/T0260A2-1c2-2.wav
(317, 8)
14617 乐器音频/传统笙C0244/C0244A2-1e5.wav
(1715, 8)
14618 乐器音频/二胡L0266/L0266A2-1c2-2.wav
(88, 8)
14619 乐器音频/编钟D0241/D0241A2-1c5-1.wav
(79, 8)
14620 乐器音频/椰胡L0288/L0288A2-1c1-2.wav
(199, 8)
14621 乐器音频/三弦T0289/T0289A2-1c2-1.wav
(181, 8)
14622 乐器音频/编铓D0298/D0298A2-1c2-1.wav
(207, 8)
14623 乐器音频/巴乌C0310/C0310A2-1d4-2.wav
(43, 8)
14624 乐器音频/洞巴C0304/C0304A2-1c1-2.wav
(259, 8)
14625 乐器音频/中阮T0260/T0260A2-1c4-2.wav
(223, 8)
14626 乐器音频/二胡L0266/L0266A2-1c1-2.wav
(150, 8)
14627 乐器音频/A调曲笛C0280/C0280A2-1c4-1.wav
(72, 8)
14628 乐器音频/葫芦琴L0314/L0314A2-1d2-2.wav
(44, 8)
14629 乐器音频/澜沧小三弦T0300/T0300A2-1c5-1.wav
(87, 8)
14630 乐器音频/洞巴C0304/C0304A2-1e1.wav
(1078, 8)
14631 乐器音频/芦笙C0307/C0307A2-1d4-1.wav
(150, 8)
14632 乐器音频/中音笙C0263/C0263A2-1c3-1.wav
(168, 8)
14633 乐器音频/傈僳族奇奔T0302/T0302A2-1c3-1.wav
(96, 8)
14634 乐器音频/傈僳族奇奔T0302/T0302A2-1b3-1.wav
(172, 8)
14635 乐器音频/埙C0283/C02

(232, 8)
14797 乐器音频/古筝T0255/T0255A2-1c2-1.wav
(104, 8)
14798 乐器音频/牛角琴L0312/L0312A2-1c5-2.wav
(160, 8)
14799 乐器音频/唢呐C0296/C0296A2-1d12-2.wav
(161, 8)
14800 乐器音频/高音键笙C0243/C0243A2-1c3-1.wav
(202, 8)
14801 乐器音频/编铓D0298/D0298A2-1c1-1.wav
(129, 8)
14802 乐器音频/唢呐C0296/C0296A2-1d12-1.wav
(156, 8)
14803 乐器音频/G调新笛C0281/C0281A2-1e2.wav
(503, 8)
14804 乐器音频/箜篌T0254/T0254A2-1c2-2.wav
(117, 8)
14805 乐器音频/二胡L0266/L0266A2-1c1-1.wav
(165, 8)
14806 乐器音频/传统笙C0244/C0244A2-1d1-2.wav
(23, 8)
14807 乐器音频/二胡L0266/L0266A2-1d9-1.wav
(97, 8)
14808 乐器音频/柳琴T0261/T0261A2-1c2-1.wav
(55, 8)
14809 乐器音频/六角高胡L0292/L0292A2-1c2-1.wav
(53, 8)
14810 乐器音频/中音板胡L0240/L0240A2-1e4.wav
(112, 8)
14811 乐器音频/唢呐C0296/C0296A2-1c2-1.wav
(56, 8)
14812 乐器音频/巴乌C0310/C0310A2-1d7-1.wav
(24, 8)
14813 乐器音频/扬琴T0267/T0267A2-1c4-1.wav
(138, 8)
14814 乐器音频/澜沧小三弦T0300/T0300A2-1e1.wav
(251, 8)
14815 乐器音频/编铓D0298/D0298A2-1d3-1.wav
(81, 8)
14816 乐器音频/葫芦丝C0309/C0309A2-1d1-2.wav
(28, 8)
14817 乐器音频/花盆鼓D0249/D0249A2-1d6-2.wav
(44, 8)
14818 乐器音频/五音排鼓D0252/D0

(112, 8)
14976 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(128, 8)
14977 乐器音频/德C0305/C0305A2-1c5-1.wav
(265, 8)
14978 乐器音频/编铓D0298/D0298A2-1c1-1.wav
(94, 8)
14979 乐器音频/柳琴T0261/T0261A2-1c4-1.wav
(91, 8)
14980 乐器音频/高音键笙C0243/C0243A2-1c4-1.wav
(220, 8)
14981 乐器音频/吐良C0308/C0308A2-1d1-2.wav
(149, 8)
14982 乐器音频/扬琴T0267/T0267A2-1c3-1.wav
(205, 8)
14983 乐器音频/傈僳族奇奔T0302/T0302A2-1d1-2.wav
(75, 8)
14984 乐器音频/扬琴T0267/T0267A2-1c4-1.wav
(141, 8)
14985 乐器音频/G调新笛C0281/C0281A2-1d5-1.wav
(72, 8)
14986 乐器音频/中阮T0260/T0260A2-1c4-1.wav
(93, 8)
14987 乐器音频/五音排鼓D0252/D0252A2-1e2.wav
(91, 8)
14988 乐器音频/高音键笙C0243/C0243A2-1d1-2.wav
(25, 8)
14989 乐器音频/傈僳族奇奔T0302/T0302A2-1c4-1.wav
(50, 8)
14990 乐器音频/六角高胡L0292/L0292A2-1c1-1.wav
(76, 8)
14991 乐器音频/德C0305/C0305A2-1c1-2.wav
(420, 8)
14992 乐器音频/编钟D0241/D0241A2-1c2-2.wav
(112, 8)
14993 乐器音频/编钟D0241/D0241A2-1c2-1.wav
(224, 8)
14994 乐器音频/二胡L0266/L0266A2-1c3-1.wav
(87, 8)
14995 乐器音频/高音板胡L0239/L0239A2-1c2-1.wav
(97, 8)
14996 乐器音频/二胡L0266/L0266A2-1c2-1.wav
(88, 8)
14997 乐器音频/碰铃D0247/D0

In [3]:
ExtractSC("test")

0 乐器音频/A调曲笛C0280/C0280A2-1c3-2.wav
(149, 8)
1 乐器音频/A调曲笛C0280/C0280A2-1d10-1.wav
(350, 8)
2 乐器音频/A调曲笛C0280/C0280A2-1c2-1.wav
(76, 8)
3 乐器音频/A调曲笛C0280/C0280A2-1c1-1.wav
(58, 8)
4 乐器音频/A调曲笛C0280/C0280A2-1c4-1.wav
(75, 8)
5 乐器音频/A调曲笛C0280/C0280A2-1d3-1.wav
(243, 8)
6 乐器音频/A调曲笛C0280/C0280A2-1e1.wav
(46, 8)
7 乐器音频/A调曲笛C0280/C0280A2-1c1-1.wav
(52, 8)
8 乐器音频/A调曲笛C0280/C0280A2-1e1.wav
(46, 8)
9 乐器音频/A调曲笛C0280/C0280A2-1c4-1.wav
(70, 8)
10 乐器音频/A调曲笛C0280/C0280A2-1d5-1.wav
(76, 8)
11 乐器音频/A调曲笛C0280/C0280A2-1d8-1.wav
(29, 8)
12 乐器音频/A调曲笛C0280/C0280A2-1e1.wav
(46, 8)
13 乐器音频/A调曲笛C0280/C0280A2-1e1.wav
(46, 8)
14 乐器音频/A调曲笛C0280/C0280A2-1b2-2.wav
(479, 8)
15 乐器音频/A调曲笛C0280/C0280A2-1e1.wav
(46, 8)
16 乐器音频/A调曲笛C0280/C0280A2-1b1-2.wav
(368, 8)
17 乐器音频/A调曲笛C0280/C0280A2-1d8-1.wav
(37, 8)
18 乐器音频/A调曲笛C0280/C0280A2-1d4-1.wav
(430, 8)
19 乐器音频/A调曲笛C0280/C0280A2-1e1.wav
(46, 8)
20 乐器音频/A调曲笛C0280/C0280A2-1c2-1.wav
(75, 8)
21 乐器音频/A调曲笛C0280/C0280A2-1d8-1.wav
(31, 8)
22 乐器音频/A调曲笛C0280/C0280A2-1c2-1.wav
(72, 8)
23 

(61, 8)
186 乐器音频/三弦T0289/T0289A2-1c4-2.wav
(39, 8)
187 乐器音频/三弦T0289/T0289A2-1d4-1.wav
(87, 8)
188 乐器音频/三弦T0289/T0289A2-1d6-1.wav
(187, 8)
189 乐器音频/三弦T0289/T0289A2-1d1-1.wav
(61, 8)
190 乐器音频/三弦T0289/T0289A2-1c3-1.wav
(119, 8)
191 乐器音频/三弦T0289/T0289A2-1c1-1.wav
(95, 8)
192 乐器音频/三弦T0289/T0289A2-1c4-2.wav
(125, 8)
193 乐器音频/三弦T0289/T0289A2-1d18-2.wav
(61, 8)
194 乐器音频/三弦T0289/T0289A2-1d5-1.wav
(200, 8)
195 乐器音频/三弦T0289/T0289A2-1c4-1.wav
(109, 8)
196 乐器音频/三弦T0289/T0289A2-1c3-2.wav
(88, 8)
197 乐器音频/三弦T0289/T0289A2-1c4-1.wav
(170, 8)
198 乐器音频/三弦T0289/T0289A2-1c1-2.wav
(104, 8)
199 乐器音频/三弦T0289/T0289A2-1d18-2.wav
(26, 8)
200 乐器音频/三弦T0289/T0289A2-1d1-1.wav
(99, 8)
201 乐器音频/三弦T0289/T0289A2-1c3-2.wav
(105, 8)
202 乐器音频/三弦T0289/T0289A2-1c2-1.wav
(173, 8)
203 乐器音频/三弦T0289/T0289A2-1c4-1.wav
(147, 8)
204 乐器音频/三弦T0289/T0289A2-1d14-1.wav
(185, 8)
205 乐器音频/三弦T0289/T0289A2-1c2-1.wav
(233, 8)
206 乐器音频/三弦T0289/T0289A2-1c2-2.wav
(168, 8)
207 乐器音频/三弦T0289/T0289A2-1c2-2.wav
(200, 8)
208 乐器音频/三弦T0289/T0289A2-1c3-

(50, 8)
373 乐器音频/中胡L0297/L0297A2-1c3-2.wav
(67, 8)
374 乐器音频/中胡L0297/L0297A2-1c5-2.wav
(50, 8)
375 乐器音频/中胡L0297/L0297A2-1c2-2.wav
(40, 8)
376 乐器音频/中胡L0297/L0297A2-1c4-2.wav
(94, 8)
377 乐器音频/中胡L0297/L0297A2-1d7-2.wav
(106, 8)
378 乐器音频/中胡L0297/L0297A2-1c1-2.wav
(130, 8)
379 乐器音频/中胡L0297/L0297A2-1c3-1.wav
(63, 8)
380 乐器音频/中胡L0297/L0297A2-1c3-2.wav
(73, 8)
381 乐器音频/中胡L0297/L0297A2-1c5-2.wav
(52, 8)
382 乐器音频/中胡L0297/L0297A2-1c3-2.wav
(73, 8)
383 乐器音频/中胡L0297/L0297A2-1d6-2.wav
(103, 8)
384 乐器音频/中胡L0297/L0297A2-1d2-2.wav
(265, 8)
385 乐器音频/中胡L0297/L0297A2-1c4-1.wav
(96, 8)
386 乐器音频/中胡L0297/L0297A2-1c1-1.wav
(112, 8)
387 乐器音频/中胡L0297/L0297A2-1e1.wav
(3707, 8)
388 乐器音频/中胡L0297/L0297A2-1d10-1.wav
(102, 8)
389 乐器音频/中胡L0297/L0297A2-1c3-2.wav
(75, 8)
390 乐器音频/中胡L0297/L0297A2-1c2-2.wav
(44, 8)
391 乐器音频/中胡L0297/L0297A2-1c3-1.wav
(70, 8)
392 乐器音频/中胡L0297/L0297A2-1d2-2.wav
(307, 8)
393 乐器音频/中胡L0297/L0297A2-1d10-2.wav
(106, 8)
394 乐器音频/中胡L0297/L0297A2-1c2-1.wav
(49, 8)
395 乐器音频/中胡L0297/L0297A2-1d5-1.wav
(

(246, 8)
562 乐器音频/中阮T0260/T0260A2-1c3-2.wav
(300, 8)
563 乐器音频/中阮T0260/T0260A2-1c2-1.wav
(181, 8)
564 乐器音频/中阮T0260/T0260A2-1c3-1.wav
(48, 8)
565 乐器音频/中阮T0260/T0260A2-1c2-1.wav
(106, 8)
566 乐器音频/中阮T0260/T0260A2-1c4-1.wav
(205, 8)
567 乐器音频/中阮T0260/T0260A2-1c4-1.wav
(181, 8)
568 乐器音频/中阮T0260/T0260A2-1c3-2.wav
(288, 8)
569 乐器音频/中阮T0260/T0260A2-1c3-1.wav
(31, 8)
570 乐器音频/中阮T0260/T0260A2-1c2-1.wav
(63, 8)
571 乐器音频/中阮T0260/T0260A2-1c2-2.wav
(335, 8)
572 乐器音频/中阮T0260/T0260A2-1c2-1.wav
(299, 8)
573 乐器音频/中阮T0260/T0260A2-1c2-1.wav
(206, 8)
574 乐器音频/中阮T0260/T0260A2-1c2-1.wav
(276, 8)
575 乐器音频/中阮T0260/T0260A2-1c2-1.wav
(143, 8)
576 乐器音频/中阮T0260/T0260A2-1c4-1.wav
(69, 8)
577 乐器音频/中阮T0260/T0260A2-1c3-2.wav
(59, 8)
578 乐器音频/中阮T0260/T0260A2-1c3-1.wav
(200, 8)
579 乐器音频/中阮T0260/T0260A2-1c4-2.wav
(194, 8)
580 乐器音频/中阮T0260/T0260A2-1c3-1.wav
(157, 8)
581 乐器音频/中阮T0260/T0260A2-1c3-2.wav
(215, 8)
582 乐器音频/中阮T0260/T0260A2-1c4-1.wav
(72, 8)
583 乐器音频/中阮T0260/T0260A2-1c4-2.wav
(312, 8)
584 乐器音频/中阮T0260/T0260A2-1c3-

(111, 8)
747 乐器音频/中音板胡L0240/L0240A2-1b3-2.wav
(179, 8)
748 乐器音频/中音板胡L0240/L0240A2-1c2-1.wav
(112, 8)
749 乐器音频/中音板胡L0240/L0240A2-1c1-1.wav
(112, 8)
750 乐器音频/中音板胡L0240/L0240A2-1d6-1.wav
(119, 8)
751 乐器音频/中音板胡L0240/L0240A2-1d1-1.wav
(76, 8)
752 乐器音频/中音板胡L0240/L0240A2-1e3.wav
(112, 8)
753 乐器音频/中音板胡L0240/L0240A2-1e4.wav
(112, 8)
754 乐器音频/中音板胡L0240/L0240A2-1b3-1.wav
(167, 8)
755 乐器音频/中音板胡L0240/L0240A2-1e5.wav
(108, 8)
756 乐器音频/中音板胡L0240/L0240A2-1d2-2.wav
(83, 8)
757 乐器音频/中音板胡L0240/L0240A2-1d2-2.wav
(88, 8)
758 乐器音频/中音板胡L0240/L0240A2-1e4.wav
(112, 8)
759 乐器音频/中音板胡L0240/L0240A2-1c3-2.wav
(113, 8)
760 乐器音频/中音板胡L0240/L0240A2-1e2.wav
(108, 8)
761 乐器音频/中音板胡L0240/L0240A2-1d10-2.wav
(592, 8)
762 乐器音频/中音板胡L0240/L0240A2-1e2.wav
(107, 8)
763 乐器音频/中音板胡L0240/L0240A2-1e1.wav
(87, 8)
764 乐器音频/中音板胡L0240/L0240A2-1e1.wav
(87, 8)
765 乐器音频/中音板胡L0240/L0240A2-1c2-1.wav
(111, 8)
766 乐器音频/中音板胡L0240/L0240A2-1e5.wav
(108, 8)
767 乐器音频/中音板胡L0240/L0240A2-1e5.wav
(108, 8)
768 乐器音频/中音板胡L0240/L0240A2-1e2.wav
(108, 8)
769 乐

...

(26, 8)
2971 乐器音频/竹排琴D0315/D0315A2-1c1-1.wav
(23, 8)
2972 乐器音频/竹排琴D0315/D0315A2-1c1-2.wav
(29, 8)
2973 乐器音频/竹排琴D0315/D0315A2-1c3-2.wav
(25, 8)
2974 乐器音频/竹排琴D0315/D0315A2-1c3-1.wav
(26, 8)
2975 乐器音频/竹排琴D0315/D0315A2-1c1-2.wav
(29, 8)
2976 乐器音频/竹排琴D0315/D0315A2-1e1.wav
(88, 8)
2977 乐器音频/竹排琴D0315/D0315A2-1e1.wav
(88, 8)
2978 乐器音频/竹排琴D0315/D0315A2-1d3-1.wav
(25, 8)
2979 乐器音频/竹排琴D0315/D0315A2-1c2-1.wav
(152, 8)
2980 乐器音频/竹排琴D0315/D0315A2-1c1-2.wav
(26, 8)
2981 乐器音频/竹排琴D0315/D0315A2-1c1-1.wav
(22, 8)
2982 乐器音频/竹排琴D0315/D0315A2-1c1-1.wav
(44, 8)
2983 乐器音频/竹排琴D0315/D0315A2-1c3-2.wav
(25, 8)
2984 乐器音频/竹排琴D0315/D0315A2-1c3-1.wav
(26, 8)
2985 乐器音频/竹排琴D0315/D0315A2-1c1-1.wav
(25, 8)
2986 乐器音频/竹排琴D0315/D0315A2-1c2-1.wav
(152, 8)
2987 乐器音频/竹排琴D0315/D0315A2-1e1.wav
(88, 8)
2988 乐器音频/竹排琴D0315/D0315A2-1c2-2.wav
(148, 8)
2989 乐器音频/竹排琴D0315/D0315A2-1c1-1.wav
(38, 8)
2990 乐器音频/竹排琴D0315/D0315A2-1d3-2.wav
(26, 8)
2991 乐器音频/竹排琴D0315/D0315A2-1c1-1.wav
(26, 8)
2992 乐器音频/竹排琴D0315/D0315A2-1c1-2.wav
(46, 8)
2993 

(63, 8)
3156 乐器音频/管子C0265/C0265A2-1e4.wav
(690, 8)
3157 乐器音频/管子C0265/C0265A2-1c4-1.wav
(43, 8)
3158 乐器音频/管子C0265/C0265A2-1c3-2.wav
(82, 8)
3159 乐器音频/管子C0265/C0265A2-1e2.wav
(211, 8)
3160 乐器音频/管子C0265/C0265A2-1c2-1.wav
(87, 8)
3161 乐器音频/管子C0265/C0265A2-1c4-2.wav
(48, 8)
3162 乐器音频/管子C0265/C0265A2-1c4-1.wav
(44, 8)
3163 乐器音频/管子C0265/C0265A2-1e2.wav
(125, 8)
3164 乐器音频/管子C0265/C0265A2-1c1-2.wav
(77, 8)
3165 乐器音频/管子C0265/C0265A2-1e2.wav
(249, 8)
3166 乐器音频/管子C0265/C0265A2-1c3-1.wav
(88, 8)
3167 乐器音频/管子C0265/C0265A2-1c3-2.wav
(81, 8)
3168 乐器音频/管子C0265/C0265A2-1e2.wav
(82, 8)
3169 乐器音频/管子C0265/C0265A2-1e2.wav
(237, 8)
3170 乐器音频/管子C0265/C0265A2-1c4-1.wav
(44, 8)
3171 乐器音频/管子C0265/C0265A2-1e3.wav
(414, 8)
3172 乐器音频/管子C0265/C0265A2-1e2.wav
(133, 8)
3173 乐器音频/管子C0265/C0265A2-1e4.wav
(287, 8)
3174 乐器音频/管子C0265/C0265A2-1e4.wav
(267, 8)
3175 乐器音频/管子C0265/C0265A2-1c4-1.wav
(48, 8)
3176 乐器音频/管子C0265/C0265A2-1e4.wav
(55, 8)
3177 乐器音频/管子C0265/C0265A2-1c3-1.wav
(77, 8)
3178 乐器音频/管子C0265/C0265A2-1c4-2.wav
(

(28, 8)
3342 乐器音频/编铓D0298/D0298A2-1c4-1.wav
(63, 8)
3343 乐器音频/编铓D0298/D0298A2-1c5-2.wav
(102, 8)
3344 乐器音频/编铓D0298/D0298A2-1d2-1.wav
(111, 8)
3345 乐器音频/编铓D0298/D0298A2-1c5-2.wav
(138, 8)
3346 乐器音频/编铓D0298/D0298A2-1c4-2.wav
(31, 8)
3347 乐器音频/编铓D0298/D0298A2-1e1.wav
(99, 8)
3348 乐器音频/编铓D0298/D0298A2-1c1-1.wav
(27, 8)
3349 乐器音频/编铓D0298/D0298A2-1c3-2.wav
(82, 8)
3350 乐器音频/编铓D0298/D0298A2-1d1-1.wav
(680, 8)
3351 乐器音频/编铓D0298/D0298A2-1d2-2.wav
(108, 8)
3352 乐器音频/编铓D0298/D0298A2-1c4-1.wav
(108, 8)
3353 乐器音频/编铓D0298/D0298A2-1e1.wav
(99, 8)
3354 乐器音频/编铓D0298/D0298A2-1c1-1.wav
(46, 8)
3355 乐器音频/编铓D0298/D0298A2-1e1.wav
(135, 8)
3356 乐器音频/编铓D0298/D0298A2-1e1.wav
(135, 8)
3357 乐器音频/编铓D0298/D0298A2-1c4-1.wav
(111, 8)
3358 乐器音频/编铓D0298/D0298A2-1c1-2.wav
(95, 8)
3359 乐器音频/编铓D0298/D0298A2-1c3-2.wav
(94, 8)
3360 乐器音频/芦笙C0307/C0307A2-1d1-2.wav
(389, 8)
3361 乐器音频/芦笙C0307/C0307A2-1d7-1.wav
(303, 8)
3362 乐器音频/芦笙C0307/C0307A2-1c1-2.wav
(485, 8)
3363 乐器音频/芦笙C0307/C0307A2-1d10-2.wav
(286, 8)
3364 乐器音频/芦笙C0307/

(216, 8)
3527 乐器音频/葫芦琴L0314/L0314A2-1d9-1.wav
(58, 8)
3528 乐器音频/葫芦琴L0314/L0314A2-1c5-2.wav
(125, 8)
3529 乐器音频/葫芦琴L0314/L0314A2-1c2-1.wav
(454, 8)
3530 乐器音频/葫芦琴L0314/L0314A2-1c4-1.wav
(276, 8)
3531 乐器音频/葫芦琴L0314/L0314A2-1c4-1.wav
(210, 8)
3532 乐器音频/葫芦琴L0314/L0314A2-1d1-1.wav
(391, 8)
3533 乐器音频/葫芦琴L0314/L0314A2-1c5-2.wav
(152, 8)
3534 乐器音频/葫芦琴L0314/L0314A2-1d2-2.wav
(43, 8)
3535 乐器音频/葫芦琴L0314/L0314A2-1d7-1.wav
(247, 8)
3536 乐器音频/葫芦琴L0314/L0314A2-1c2-1.wav
(402, 8)
3537 乐器音频/葫芦琴L0314/L0314A2-1d5-2.wav
(212, 8)
3538 乐器音频/葫芦琴L0314/L0314A2-1d3-1.wav
(22, 8)
3539 乐器音频/葫芦琴L0314/L0314A2-1d6-1.wav
(55, 8)
3540 乐器音频/葫芦琴L0314/L0314A2-1d7-2.wav
(235, 8)
3541 乐器音频/葫芦琴L0314/L0314A2-1c2-1.wav
(445, 8)
3542 乐器音频/葫芦琴L0314/L0314A2-1d9-1.wav
(53, 8)
3543 乐器音频/葫芦琴L0314/L0314A2-1c3-1.wav
(212, 8)
3544 乐器音频/葫芦琴L0314/L0314A2-1d4-2.wav
(572, 8)
3545 乐器音频/葫芦琴L0314/L0314A2-1b2-1.wav
(255, 8)
3546 乐器音频/铙D0269/D0269A2-1d5-2.wav
(239, 8)
3547 乐器音频/铙D0269/D0269A2-1b2-2.wav
(201, 8)
3548 乐器音频/铙D0269/D0269A2-1d2-1.wav

(93, 8)
3710 乐器音频/高音键笙C0243/C0243A2-1c4-1.wav
(211, 8)
3711 乐器音频/高音键笙C0243/C0243A2-1c3-1.wav
(220, 8)
3712 乐器音频/高音键笙C0243/C0243A2-1c4-1.wav
(209, 8)
3713 乐器音频/高音键笙C0243/C0243A2-1c1-2.wav
(115, 8)
3714 乐器音频/高音键笙C0243/C0243A2-1d7-1.wav
(1268, 8)
3715 乐器音频/高音键笙C0243/C0243A2-1c2-2.wav
(172, 8)
3716 乐器音频/高音键笙C0243/C0243A2-1c3-1.wav
(173, 8)
3717 乐器音频/高音键笙C0243/C0243A2-1c3-2.wav
(220, 8)
3718 乐器音频/高音键笙C0243/C0243A2-1c2-1.wav
(138, 8)
3719 乐器音频/高音键笙C0243/C0243A2-1d6-2.wav
(1292, 8)
3720 乐器音频/高音键笙C0243/C0243A2-1c2-1.wav
(147, 8)
3721 乐器音频/高音键笙C0243/C0243A2-1d2-1.wav
(636, 8)
3722 乐器音频/高音键笙C0243/C0243A2-1c5-2.wav
(145, 8)
3723 乐器音频/高音键笙C0243/C0243A2-1c4-1.wav
(211, 8)
3724 乐器音频/高音键笙C0243/C0243A2-1c4-2.wav
(173, 8)
3725 乐器音频/高音键笙C0243/C0243A2-1d1-1.wav
(31, 8)
3726 乐器音频/高音键笙C0243/C0243A2-1c2-2.wav
(152, 8)
3727 乐器音频/高音键笙C0243/C0243A2-1c3-1.wav
(243, 8)
3728 乐器音频/高音键笙C0243/C0243A2-1c3-2.wav
(226, 8)
3729 乐器音频/高音键笙C0243/C0243A2-1c1-1.wav
(155, 8)
3730 乐器音频/高音键笙C0243/C0243A2-1c5-1.wav
(152, 8)
3731